# Тематическое моделирование

Я честно не придумала, на каком корпусе можно протестировать то, что у меня получится, и решила оставить корпус фантастики. Вместо этого у меня появилась гипотеза для метрики по определению количества топиков.

In [ ]:
# а то забуду
random_seed = 42

## Предобработка

Оставлю только существительные, прилагательные, глаголы и наречия, остальное выброшу — от местоимений не очень много смысла в топик моделлинге.

In [ ]:
import os

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from tqdm import tqdm

Особенности под эту задачу:

* вместо списка стоп-слов из NLTK я взяла тот, что лежал в репозитории курса: там есть всякие местоимения-прилагательные типа «весь», которые сильно мешались в прошлых вариациях;

* вместо всех слов оставляю только существительные, прилагательные, глаголы и наречия;

* дополнительно проверяю все токены, не имеют ли они отношения к имени (у pymorphy есть для этого специальные теги), чтобы тоже не замусоривало корпус.

In [3]:
morph = MorphAnalyzer()
with open("../data/stop_ru.txt", "r", encoding="utf-8") as f:
    stops = set([line.strip("\n") for line in f.readlines()])

In [4]:
meaningful_pos = set(["NOUN", "ADJF", "ADJS", "COMP", "VERB", "INFN", "ADVB", "PRED"])
names_tags = set(["Name", "Surn", "Patr"])

def preprocess_text(text, stops, morph):
    lemmas_list = []
    for token in simple_word_tokenize(text):
        token_analysis = morph.parse(token)[0]
        if token_analysis.tag.POS in meaningful_pos:
            if (token_analysis.normal_form not in stops) \
            and (names_tags not in token_analysis.tag):
                lemmas_list.append(token_analysis.normal_form)
    text_lemmatized = " ".join(lemmas_list)
    return text_lemmatized

In [5]:
path_to_corpus = "../data/fantasy_corpus"
processed_texts = []
not_parsed = []

for item in tqdm([fn for fn in os.listdir(path_to_corpus) if fn.endswith(".txt")]):
    with open(os.path.join(path_to_corpus, item), "r", encoding="utf-8") as f:
        try:
            raw_text = f.read()
            processed_texts.append(preprocess_text(raw_text, stops, morph))
        except UnicodeDecodeError:
            not_parsed.append(item)
print("Не обработались:\n{}".format("\n".join(not_parsed)))

100%|██████████| 341/341 [1:01:29<00:00, 10.82s/it]

Не обработались:
1969_Zabelin_Zapiski_hronoskopista.txt
1989_Glazkov_Vtoroj_spisok.txt
2008_Akhmanov_Zaklinatel_dzhinnov.txt
1977_Snegov_Kol_co_obratnogo_vremeni.txt
2006_Romanov_Vystrel_v_zerkalo.txt
1965_Varshavskij_Pod_nogami_Zemlya.txt
1967_Varshavskij_Lavka_snovidenij.txt
1987_Suhanov_Avatara.txt
2012_Bachilo_Ne_nuzhny.txt
1960_Zhuravlyova_Skvoz__vremya.txt
2008_Assiriyskie_tanki_u_vrat_Memfisa.txt
2002_Moshkov_Pobeda_uskolzaet.txt
1924_Goncharov_Psihomashina.txt
2009_Bachilo_Moskovskiy_okhotnik.txt
1967_Pavlov_Korona_solnca.txt
2008_Akhmanov_Skify_piruyut_na_zakate.txt
2004_Bachilo_Nakhta.txt
1930_Palej_Planeta_Kim.txt
1959_Zabelin_V_pogone_za_ihtiozavrami.txt
1993_Moshkov_Vozvrashchenie_iz_otpuska.txt
1971_Zhuravlyova_Snezhnyj_most_nad_propast_yu.txt
2007_Bachilo_Brigadir.txt
2002_Nikolaev_Relikt.txt
1969_Mirer_U_menia_deviat_zhiznej.txt
1963_Varshavskij_Molekulyarnoe_kafe.txt
1989_Glazkov_Bezdomnye_skital_cy.txt
1997_Shumil_K_voprosu_o_smysle_zhizni.txt
1987_Bachilo_Pomoch_mozh

Посмотрим, сколько у нас всего текстов:

In [6]:
len(processed_texts)

296

И сколько токенов:

In [7]:
"{:,}".format(sum([len(text.split()) for text in processed_texts]))

'7,841,061'

## TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfidf_vec = TfidfVectorizer()
tfidf_m = tfidf_vec.fit_transform(processed_texts)
words = tfidf_vec.get_feature_names()

## Обучение и выбор числа топиков

**Идея:** я хочу попробовать оттолкнуться от семантической близости слов в топике. Если топик выделился осознанно, то семантическая близость первых 20\* слов будет высокой.

Сейчас (до того, как я начала писать код и проверять эту гипотезу) есть одно «но» — в топик могут входить не только синонимы, но и антонимы (например, слова «дорогой» и «дешёвый» вполне могут вместе попасть в финансовый топик), семантическая близость которых равна -1; тем не менее, кажется, что всё-таки даже если и так, пара антонимов не так сильно утянет значение средней семантической близости вниз, как набор слов в стиле «стул, лошадь, 23».

_\* почему 20? Если мы говорим о «авторитарных» топиках, то кажется, что такое число может отразить и важные слова с большим весом, и менее важные с весом поменьше; в случае с «демократическими» топиками 20 — достаточное число слов, чтобы зацепить разные аспекты, которые могут быть в топике._

### Проверка топиков

Я буду использовать модель с такими параметрами:

* обучена на текстах НКРЯ образца 2019 года,

* 270 миллионов слов, объём словаря — 189 193 слова.

In [10]:
import gensim
import zipfile

In [11]:
w2v_model_path = "../data/ruscorpora_upos_cbow_300_20_2019.zip"
with zipfile.ZipFile(w2v_model_path, "r") as archive:
    stream = archive.open("model.bin")
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Как считается семантическая близость _слов в топике_: 

1. составляются все возможные пары из слов, 

2. для каждой считается сем. близость,

3. берётся среднее значение всех полученных величин.

In [12]:
from itertools import combinations
from statistics import mean

In [13]:
def assess_topic(topic_words, model):
    sem_similarities = []
    for words_pair in combinations(topic_words, 2):
        try:
            word1 = "{}_{}".format(words_pair[0], 
                                   morph.parse(words_pair[0])[0].tag.POS)
            word2 = "{}_{}".format(words_pair[1], 
                                   morph.parse(words_pair[1])[0].tag.POS)
            sim = model.similarity(word1, word2)
        except:
            sim = 0
        sem_similarities.append(sim)
    return mean(sem_similarities)

In [14]:
def assess_trained_lda(all_topic_words, model):
    all_topic_scores = []
    for this_topic_words in all_topic_words:
        this_topic_score = assess_topic(this_topic_words, model)
        all_topic_scores.append(this_topic_score)
    return all_topic_scores

### Обучение модели

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

In [16]:
def fit_lda(feature_m, num_topics, words):
    # обучаем модель
    lda = LatentDirichletAllocation(n_components=num_topics,
                                    learning_method="online",
                                    random_state=random_seed)
    lda = lda.fit(feature_m)
    # забираем топ-20 слов топика
    num_top_words = 20
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topic_words = [words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        topics.append(topic_words)
    return lda, topics

Разброс топиков сделаю на порядок — от 5 до 50.

_почему 5? не знаю, просто захотелось 5…_

In [20]:
%%time
experiment_scores = {}
experiment_scores_all = {}
experiment_topics = {}

for topic_num in range(5, 51):
    lda_trained, all_topics_words = fit_lda(tfidf_m, topic_num, words)
    all_topics_scores = assess_trained_lda(all_topics_words, model)
    experiment_scores_all[topic_num] = all_topics_scores
    experiment_scores[topic_num] = mean(all_topics_scores)
    experiment_topics[topic_num] = all_topics_words

CPU times: user 1h 13min 26s, sys: 11min 36s, total: 1h 25min 3s
Wall time: 24min 23s


Все результаты сложу в отдельный файл, чтобы тетрадка не превратилась в портянку:

In [21]:
import json

In [22]:
with open("./topic-modelling_topics.txt", "w", encoding="utf-8") as file_topics:
    for topic_num in range(5, 51):
        file_topics.write("{}\n".format(topic_num))
        for topic in experiment_topics[topic_num]:
            file_topics.write("{}\n".format(", ".join(topic)))

## Результаты

Посмотрим на результат работы модели с лучшей метрикой:

In [23]:
best_score_topic_num = max(experiment_scores, key=experiment_scores.get)
print("Лучшее по сем. близости количество тем: {}, результат: {:.5f}".format(best_score_topic_num,
                                                                            experiment_scores[best_score_topic_num]))

Лучшее по сем. близости количество тем: 8, результат: 0.01376


Результат, если честно, не очень впечатляет…

In [24]:
lda_trained, all_topics_words = fit_lda(tfidf_m, best_score_topic_num, words)

In [25]:
for i, topic in enumerate(all_topics_words):
    print("topic no: {:2}\twords: {}".format(i, ", ".join(topic)))

topic no:  0	words: боячек, ариэль, майк, несправедливость, олег, валентайна, акраб, 25ь3ый, макбет, мускулус, коллекторский, загорланить, геномодель, максик, вузина, проворонить, хорог, шлюз, флигранно, ксенофонт
topic no:  1	words: андрей, горелов, скотенков, форам, то, вечеровский, оказаться, валентин, учреждаться, открытый, малян, лицо, талреп, гуща, крутнувшийся, макс, просчитать, мудь, тридцатисемилетний, большой
topic no:  2	words: никки, шурка, то, мюргита, рука, стив, румат, земля, глаз, максим, джон, ладушкин, кирилл, апостол, психолог, арсен, ружейный, знать, солнце, большой
topic no:  3	words: павлыш, клэйтон, марат, игорь, перхуш, странник, бартон, доктор, барнаби, рука, брошюрный, ассасин, сериза, ротан, арбалетный, параллельно, противостояние, то, крыловой, позадавать
topic no:  4	words: волгин, стать, то, рэсся, самый, дело, знать, эверс, амит, родис, рифт, большой, крэл, стис, прыгнуть, вести, рука, друг, должный, высвобождение
topic no:  5	words: кора, фролов, глаз, с

Не очень похоже на внятные результаты… а если попробовать минимум?

In [26]:
worst_score_topic_num = min(experiment_scores, key=experiment_scores.get)
print("Кол-во тем, где сем. близость минимальна: {}, результат: {:.5f}".format(worst_score_topic_num,
                                                                            experiment_scores[worst_score_topic_num]))
lda_trained, all_topics_words = fit_lda(tfidf_m, worst_score_topic_num, words)
for i, topic in enumerate(all_topics_words):
    print("topic no: {:2}\twords: {}".format(i, ", ".join(topic)))

Кол-во тем, где сем. близость минимальна: 32, результат: 0.00469
topic no:  0	words: ванюшка, конобей, таям, боячек, нааля, малян, гузик, несправедливость, валентайна, акраб, 25ь3ый, макбет, бартон, мускулус, коллекторский, гесера, загорланить, геномодель, максик, вузина
topic no:  1	words: шурка, андрей, учреждаться, белопольский, олег, оказаться, крутнувшийся, гуща, открытый, просчитать, мудь, тридцатисемилетний, шед, шахерзад, детдомовский, закаркать, рымшёныш, кора, рекомендация, папандопулос
topic no:  2	words: олег, солнце, земля, психолог, ружейный, оперчасть, комингс, суккулента, андрей, доменико, хээхо, тратаниана, бульшуя, ирка, туманный, высоконогий, брайль, тарогойя, перс, буфетчица
topic no:  3	words: странник, брошюрный, лось, арбалетный, параллельно, противостояние, крыловой, позадавать, партийка, подмес, ответный, переменность, полэкран, ихтиандр, шуля, архимед, уоттер, переплавлятьный, оптоэлектронный, оставление
topic no:  4	words: то, рука, стать, знать, самый, больш

Всё равно не то чтобы были осмысленные топики. Наверное, это связано с тем, что в фантастике много всяких несуществующих слов — да и имён осталось достаточно… наверное, такие несуществующие слова тоже нужно было убрать.

_Бонус:_ ничто не ново под луной — семантику прикручивали ещё в 2013, но чуть иначе — [сначала считали усреднённый вектор-центроид топика, потом считали косинусную близость каждого слова в топике к этому центроиду](https://www.researchgate.net/publication/235974307_Evaluating_Topic_Coherence_Using_Distributional_Semantics)), но результат был сравнительно приятный — цифрмы от более принятой метрики PMI там приятные. Единственное «но» — данные там были не литературные, а газетные (20 Newsgroups и прочая классика жанра для классификации) — там тоже будет проблема с именами, но нет проблемы с несуществующей лексикой.

## Что ещё можно было бы попробовать сделать?

Пока я писала плюсы и минусы своей идеи, мне пришло в голову, что тут хорошо бы справилась какая-нибудь онтология или ворднет — благодаря тому, что в них реализована идея уровневости; можно было бы смотреть, совпадают ли у слов в топиках гиперонимы.